In [2]:
# IMPORTS
import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

In [3]:

import requests
from io import StringIO

def get_url_info(url) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    # url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for withdrawn ipos.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [4]:
withdrawn_ipos = get_url_info(url="https://stockanalysis.com/ipos/withdrawn/")
withdrawn_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [5]:
def get_company_class(company_name):
    if pd.isna(company_name):
        return 'Other'
    
    company_name = company_name.lower()
    
    if 'acquisition corp' in company_name or 'acquisition corporation' in company_name:
        return 'Acq.Corp'
    elif 'inc' in company_name or 'incorporated' in company_name:
        return 'Inc'
    elif 'group' in company_name:
        return 'Group'
    elif 'holdings' in company_name:
        return 'Holdings'
    elif 'ltd' in company_name or 'limited' in company_name:
        return 'Ltd'
    else:
        return 'Other'

withdrawn_ipos['Company Class'] = withdrawn_ipos['Company Name'].apply(get_company_class)

In [6]:
def get_avg_price(price_range) -> float:
    if pd.isna(price_range) or price_range == '-':
        return None
    
    # Remove '$' and split on '-' to get range bounds
    price_range = price_range.replace('$', '')
    prices = price_range.split('-')
    
    # Convert prices to float
    prices = [float(p) for p in prices]
    
    # If single price, return it
    if len(prices) == 1:
        return prices[0]
    
    # Otherwise return average of range
    return sum(prices) / len(prices)

withdrawn_ipos['Avg. Price'] = withdrawn_ipos['Price Range'].apply(get_avg_price)

withdrawn_ipos.describe(include='all')


,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
count,100,100,100,100,100,73.000000
unique,100,100,30,48,6,NaN
top,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
freq,1,1,27,28,51,NaN
mean,NaN,NaN,NaN,NaN,NaN,7.918493
std,NaN,NaN,NaN,NaN,NaN,3.882649
min,NaN,NaN,NaN,NaN,NaN,2.250000
25%,NaN,NaN,NaN,NaN,NaN,5.000000
50%,NaN,NaN,NaN,NaN,NaN,6.500000
75%,NaN,NaN,NaN,NaN,NaN,10.000000


In [7]:
withdrawn_ipos

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN


In [8]:
# Q1 which company class has the highest withdrawn value?
if withdrawn_ipos['Shares Offered'].dtype == 'object':
    withdrawn_ipos['Shares Offered'] = withdrawn_ipos['Shares Offered'].str.replace('-', 'NaN').astype(float)
    
    
sum(~withdrawn_ipos['Shares Offered'].isna())


withdrawn_ipos['Withdrawn Value'] = withdrawn_ipos['Avg. Price'] * withdrawn_ipos['Shares Offered']
sum(~withdrawn_ipos['Withdrawn Value'].isna())

total_withdrawn_value_by_class = withdrawn_ipos.groupby("Company Class")["Withdrawn Value"].sum().sort_values(ascending=False)

print(total_withdrawn_value_by_class)
print(f"highest withdrawn value: {total_withdrawn_value_by_class.index[0]}    {total_withdrawn_value_by_class[0]:.2e}")



Company Class
Acq.Corp    4.021000e+09
Inc         2.257164e+09
Other       7.679200e+08
Ltd         3.217346e+08
Holdings    3.030000e+08
Group       3.378750e+07
Name: Withdrawn Value, dtype: float64
highest withdrawn value: Acq.Corp    4.02e+09


In [9]:
ipos_2024=get_url_info(url="https://stockanalysis.com/ipos/2024/")

In [12]:
if ipos_2024['IPO Date'].dtype == 'object':
    ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'])

ipos_2024_first5months=ipos_2024[ipos_2024['IPO Date'].dt.month <= 5]

print(len(ipos_2024_first5months))


77


In [13]:
ipos_2024_first5months.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$11.60,-
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$34.92,103.21%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$21.00,7.36%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.52,5.00%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.32,-93.04%


In [14]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ipos_2024_first5months['Symbol']):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-5) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)
     

0 NAKA
1 BOW
2 HDL
3 RFAI
4 JDZG
5 RAY
6 BTOC
7 ZK
8 GPAT
9 PAL
10 SVCO
11 NNE
12 CCIX
13 VIK
14 ZONE
15 LOAR
16 MRX
17 RBRK
18 NCI
19 MFI
20 YYGH
21 TRSG
22 CDTG
23 CTRI
24 IBTA
25 MTEN
26 SUPX
27 TWG
28 ULS
29 PACS
30 MNDR
31 CTNM
32 MAMO
33 ZBAO
34 BOLD
35 MMA
36 UBXG
37 IBAC
38 AUNA
39 BKHA
40 LOBO
41 RDDT
42 ALAB
43 INTJ
44 RYDE
45 LGCL
46 SMXT
47 VHAI
48 DYCQ
49 CHRO
50 UMAC
51 HLXB
52 MGX
53 TBBB
54 TELO
55 KYTX
56 PMNT
57 AHR
58 LEGT
59 ANRO
60 GUTS
61 AS
62 FBLG
63 AVBP
64 BTSG
65 HAO
66 CGON
67 YIBO
68 JL
69 SUGP
70 JVSA
71 KSPI
72 CCTG
73 PSBD
74 SYNX
75 SDHC
76 ROMA


In [15]:
# Add 252d growth column using pandas shift
stocks_df['growth_252d'] = stocks_df.groupby('Ticker')['Close'].transform(lambda x: x / x.shift(252))


In [16]:

stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [17]:
stocks_df['Date'].head()


0    2024-05-31
1    2024-06-03
2    2024-06-04
3    2024-06-05
4    2024-06-06
Name: Date, dtype: object

In [18]:
# Q2 what is the median sharpe ratio for the month of June 2025 for companies that went public in the first 5 months of 2024?
date_str = "2025-06-06"
stocks_df[stocks_df['Date'].astype('str')== date_str][['Sharpe','growth_252d']].agg(['count', 'mean', 'std', 'min', lambda x: x.quantile(0.25), 'median', lambda x: x.quantile(0.75), 'max']).round(6)

,Sharpe,growth_252d
count,73.000000,73.000000
mean,0.297523,1.227948
std,0.523190,1.480237
min,-0.079677,0.024970
<lambda>,0.040265,0.293510
median,0.083768,0.763188
<lambda>,0.331967,1.446667
max,2.835668,8.097413


In [19]:
AA=stocks_df[stocks_df['Date'].astype('str')== date_str].sort_values(by='Sharpe', ascending=False)
BB=stocks_df[stocks_df['Date'].astype('str')== date_str].sort_values(by='growth_252d', ascending=False)


CC=pd.concat([
    AA[['Ticker', 'Sharpe', 'growth_252d']].rename(columns={'Sharpe': 'Sharpe_AA', 'growth_252d': 'growth_252d_AA'}),
    BB[['Ticker', 'Sharpe', 'growth_252d']].rename(columns={'Sharpe': 'Sharpe_BB', 'growth_252d': 'growth_252d_BB'})
], axis=1).drop_duplicates()

CC.head(10)

,Ticker,Sharpe_AA,growth_252d_AA,Ticker,Sharpe_BB,growth_252d_BB
11178,BKHA,2.835668,1.045881,BKHA,2.835668,1.045881
21224,JVSA,2.041531,1.071076,JVSA,2.041531,1.071076
17139,LEGT,1.940267,1.049407,LEGT,1.940267,1.049407
10603,IBAC,1.637119,1.044611,IBAC,1.637119,1.044611
5077,NCI,1.181375,0.572687,NCI,1.181375,0.572687
14831,HLXB,1.123493,1.061404,HLXB,1.123493,1.061404
8529,MNDR,0.974234,1.030769,MNDR,0.974234,1.030769
13838,DYCQ,0.969321,1.059863,DYCQ,0.969321,1.059863
12402,INTJ,0.744512,0.661386,INTJ,0.744512,0.661386
20564,JL,0.566222,8.097413,JL,0.566222,8.097413


In [21]:
num_months=range(1,12,1)
for i in num_months:
    stocks_df[f'future_growth_{i}m']=stocks_df['Close'].shift(-i)/stocks_df['Close']
len(stocks_df['Ticker'].unique()) 

77

In [22]:
min_date = stocks_df.groupby('Ticker').agg({'Date': 'first',}).reset_index()

# Display all rows instead of just first 5
min_date_joined_stocks_df=min_date.merge(stocks_df, on=['Ticker', 'Date'], how='inner')


In [23]:

# Get all columns that start with 'future'
future_columns = [col for col in min_date_joined_stocks_df.columns if col.startswith('future')]
min_date_joined_stocks_df_stats=min_date_joined_stocks_df[['Date','Ticker']+future_columns].describe()



In [96]:
# Q3 What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth? the sortedfuture growth sorted by the mean
min_date_joined_stocks_df_stats.loc['mean',future_columns].sort_values(ascending=False)


future_growth_3m     1.078582
future_growth_2m     1.063696
future_growth_4m     1.047177
future_growth_6m     1.043419
future_growth_7m     1.039056
future_growth_5m     1.038392
future_growth_9m     1.036341
future_growth_8m     1.031936
future_growth_11m    1.014684
future_growth_10m    1.010579
future_growth_1m     1.007921
Name: mean, dtype: float64

In [25]:
import gdown
import os
if not os.path.exists("data.parquet"):
    file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
    gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")


In [26]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]


In [28]:
# Q4  What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?
net_income = 1000 * (selected_df['growth_future_30d'] - 1).mean()
print(f"Net income: {net_income:.2f}")


Net income: 24295.52
